# Install libraries

In [4]:
!pip install bs4
!pip install requests
!pip install lxml
!pip install html5lib

from bs4 import BeautifulSoup
import requests
import pandas as pd

  Stored in directory: /home/dsxuser/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


# Extract table from webpage

In [5]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

my_table = BeautifulSoup(source , features = "lxml").find('table').prettify()


my_table = pd.read_html(my_table)
my_table = pd.DataFrame(my_table[0], columns = ['Postcode', 'Borough', 'Neighbourhood'])
my_table.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Drop 'Not assigned' Boroughs

In [24]:
borough_not_assigned = my_table[my_table['Borough'] == "Not assigned"].index
my_table.drop(borough_not_assigned, inplace = True)
my_table.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


# Concatonate Neighbourhoods to same postcode

In [7]:
my_table_concat = my_table.groupby(['Postcode', 'Borough'], as_index = False).agg({'Neighbourhood': ','.join})
my_table_concat.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Find Neighbourhoods that are 'Not assigned' and change

In [27]:
neigh_not_assigned = my_table_concat[my_table_concat['Neighbourhood'] == "Not assigned"].index
print(neigh_not_assigned)                                #Find positions where Neighbour is not assigned
print(my_table_concat.loc[[93]])                         #Check
my_table_concat.ix[93, 'Neighbourhood'] = "Queen's Park" #Change manually
print(my_table_concat.loc[[93]])                         #Check to see if it updated

Int64Index([], dtype='int64')
   Postcode       Borough Neighbourhood
93      M9A  Queen's Park  Queen's Park
   Postcode       Borough Neighbourhood
93      M9A  Queen's Park  Queen's Park


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


# Print size

In [22]:
my_table_concat.size

309